In [11]:
import string
import random
import numpy as np

In [6]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('senseval')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package senseval to /root/nltk_data...
[nltk_data]   Package senseval is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords, senseval
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# --- Функція попередньої обробки тексту (без змін) ---
def process_text(text_tokens):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = []
    for token in text_tokens:
        token = token.lower()
        if token.isalpha() and token not in stop_words:
            stemmed_token = stemmer.stem(token)
            cleaned_tokens.append(stemmed_token)
    return cleaned_tokens

# --- Реалізація Наївного Баєсового класифікатора (без змін) ---
def build_freqs(texts, labels):
    freqs = {}
    for y, text in zip(labels, texts):
        for word in process_text(text):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs

def train_naive_bayes(freqs, train_y):
    loglikelihood = {}
    logprior = 0
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    N_pos = N_neg = 0
    for pair, count in freqs.items():
        if pair[1] == 1:
            N_pos += count
        else:
            N_neg += count
    D = len(train_y)
    D_pos = np.sum(train_y)
    D_neg = D - D_pos
    logprior = np.log(D_pos) - np.log(D_neg)
    for word in vocab:
        freq_pos = freqs.get((word, 1.0), 0)
        freq_neg = freqs.get((word, 0.0), 0)
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)
    return logprior, loglikelihood

def naive_bayes_predict(text_tokens, logprior, loglikelihood):
    word_l = process_text(text_tokens)
    p = logprior
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    y_hats = []
    for text in test_x:
        if naive_bayes_predict(text, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
    error = np.mean(np.abs(np.array(y_hats) - np.array(test_y)))
    accuracy = 1 - error
    return accuracy


In [14]:
# 1. Завантаження даних з корпусу senseval (ВИПРАВЛЕНА ВЕРСІЯ)
print("\nКрок 1: Завантаження та підготовка даних з 'senseval'...")
documents = []

# Функція для витягнення слів з контексту (ВИПРАВЛЕНА ВЕРСІЯ)
def extract_words_from_context(context):
    # ВИПРАВЛЕННЯ ТУТ: Беремо перший елемент з кожного кортежу,
    # незалежно від його довжини.
    return [item[0] for item in context]

for inst in senseval.instances('hard.pos'):
    context_words = extract_words_from_context(inst.context)
    documents.append((context_words, 0))

for inst in senseval.instances('interest.pos'):
    context_words = extract_words_from_context(inst.context)
    documents.append((context_words, 1))

random.shuffle(documents)

print(f"Завантажено {len(documents)} речень.")
print("Приклад речення (для слова 'hard' або 'interest'):")
print(f"Текст: {' '.join(documents[0][0])}")
print(f"Мітка класу: {documents[0][1]} (0=hard, 1=interest)")


# 2. Розділення на тренувальний та тестовий набори
print("\nКрок 2: Розділення даних...")
split_ratio = int(len(documents) * 0.8)
train_documents = documents[:split_ratio]
test_documents = documents[split_ratio:]

train_x = [doc[0] for doc in train_documents]
train_y = [doc[1] for doc in train_documents]

test_x = [doc[0] for doc in test_documents]
test_y = [doc[1] for doc in test_documents]
print(f"Розмір тренувального набору: {len(train_x)} речень.")
print(f"Розмір тестового набору: {len(test_x)} речень.")


# 3. Побудова частотного словника
print("\nКрок 3: Побудова частотного словника...")
freqs = build_freqs(train_x, train_y)
print(f"Словник побудовано. Кількість унікальних пар ('слово', мітка): {len(freqs)}")


# 4. Навчання моделі
print("\nКрок 4: Навчання моделі...")
logprior, loglikelihood = train_naive_bayes(freqs, train_y)
print("Модель навчено.")


Крок 1: Завантаження та підготовка даних з 'senseval'...
Завантажено 6701 речень.
Приклад речення (для слова 'hard' або 'interest'):
Текст: now their view is one of misery -- hard earth blanketed with discarded water bottles and human waste , weary children in tattered clothes lining up for rations .
Мітка класу: 0 (0=hard, 1=interest)

Крок 2: Розділення даних...
Розмір тренувального набору: 5360 речень.
Розмір тестового набору: 1341 речень.

Крок 3: Побудова частотного словника...
Словник побудовано. Кількість унікальних пар ('слово', мітка): 11614

Крок 4: Навчання моделі...
Модель навчено.


In [15]:
# 5. Оцінка точності
print("\nКрок 5: Оцінка точності на тестовому наборі...")
accuracy = test_naive_bayes(test_x, test_y, logprior, loglikelihood)
print(f"\nТочність класифікатора: {accuracy:.4f} ({accuracy:.2%})")


# 6. Тестування на власному прикладі
print("\nКрок 6: Тестування на власному реченні...")
test_sentence_1 = "This is a very hard task".split()
test_sentence_2 = "I have no interest in this topic".split()

pred1 = naive_bayes_predict(test_sentence_1, logprior, loglikelihood)
pred2 = naive_bayes_predict(test_sentence_2, logprior, loglikelihood)

def decode_prediction(score):
    return "'interest'" if score > 0 else "'hard'"

print(f"Речення: {' '.join(test_sentence_1)}")
print(f"Прогноз: речення належить до класу {decode_prediction(pred1)}")
print("-" * 20)
print(f"Речення: {' '.join(test_sentence_2)}")
print(f"Прогноз: речення належить до класу {decode_prediction(pred2)}")


Крок 5: Оцінка точності на тестовому наборі...

Точність класифікатора: 0.9806 (98.06%)

Крок 6: Тестування на власному реченні...
Речення: This is a very hard task
Прогноз: речення належить до класу 'hard'
--------------------
Речення: I have no interest in this topic
Прогноз: речення належить до класу 'interest'


In [16]:
import json

# Перевіряємо, чи існують наші навчені параметри
if 'logprior' in locals() and 'loglikelihood' in locals():

    # Створюємо словник для експорту
    model_to_export = {
        'logprior': logprior,
        'loglikelihood': loglikelihood
    }

    # Зберігаємо словник у файл model.json
    with open('model.json', 'w', encoding='utf-8') as f:
        json.dump(model_to_export, f, ensure_ascii=False, indent=4)

    print("Модель успішно експортована у файл 'model.json'")
    # Тепер ви можете завантажити цей файл зі свого середовища (напр., з Google Colab)
else:
    print("Помилка: змінні 'logprior' та 'loglikelihood' не знайдено. Будь ласка, навчіть модель перед експортом.")

Модель успішно експортована у файл 'model.json'
